# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import requests
import time
from pprint import pprint
from scipy.stats import linregress
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression


# Import API key
from open_weather_key import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "city.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

619

In [3]:
len(cities)

619

In [4]:
sub_cities = cities[0:10]
sub_cities

['vaini',
 'lakes entrance',
 'bethel',
 'kapaa',
 'ushuaia',
 'babati',
 'staraya toropa',
 'castro',
 'turkistan',
 'tiznit']

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [7]:
#init lists to hold parsed data
lats = []
longs = []
temps = []
feel_likes = []
humids = []
pressures = []
speeds = []
cloudiness = []
cities_found = []

for i in range(len(cities)):
    #get the city
    city = cities[i]
    
    #create the url
    units = "imperial"
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&units={units}&appid={weather_api_key}"
#     print(url)

    #make the request
    response = requests.get(url)
    print(response.status_code)
    
    #error check
    if response.status_code == 200:
        data = response.json()
#         pprint(data)
        
        try:
            #extract the data
            lat = data["coord"]["lat"]
            long = data["coord"]["lon"]
            temp = data["main"]["temp"]
            feels_like = data["main"]["feels_like"]
            humidity = data["main"]["humidity"]
            pressure = data["main"]["pressure"]
            speed = data["wind"]["speed"]
            clouds = data["clouds"]["all"]
            city_name = data["name"]

            #save the data
            lats.append(lat)
            longs.append(long)
            temps.append(temp)
            feel_likes.append(feels_like)
            humids.append(humidity)
            pressures.append(pressure)
            speeds.append(speed)
            cloudiness.append(clouds)
            cities_found.append(city_name)
        except Exception as e:
            print(f"Through exception for city {city}: {e}")
    
    elif response.status_code == 404:
        print(f"Missing data in OpenWeatherAPI for {city}")
    else:
        print("Oh gosh darnit. The API is broken. Sad Face.")
        
    #print every 5
    if (i % 5 == 0):
        print(f"Got data for city index: {i} of {len(cities)}")
    
    #sleep for requests
    time.sleep(1)
        
# make the dataframe
df = pd.DataFrame()
df["City"] = cities_found
df["Latitude"] = lats
df["Longitude"] = longs
df["Temperature"] = temps
df["Feels Like"] = feel_likes
df["Humidity"] = humids
df["Pressure"] = pressures
df["Wind Speed"] = speeds
df["Cloudiness"] = cloudiness

df.head()

200
Got data for city index: 0 of 619
200
200
200
200
200
Got data for city index: 5 of 619
200
200
200
200
200
Got data for city index: 10 of 619
404
Missing data in OpenWeatherAPI for palabuhanratu
200
200
200
200
Got data for city index: 15 of 619
200
200
200
200
200
Got data for city index: 20 of 619
200
200
404
Missing data in OpenWeatherAPI for sinkat
200
200
Got data for city index: 25 of 619
200
200
200
200
200
Got data for city index: 30 of 619
200
200
200
200
200
Got data for city index: 35 of 619
404
Missing data in OpenWeatherAPI for belushya guba
200
200
200
200
Got data for city index: 40 of 619
200
200
200
200
200
Got data for city index: 45 of 619
200
404
Missing data in OpenWeatherAPI for kamenskoye
404
Missing data in OpenWeatherAPI for illoqqortoormiut
200
200
Got data for city index: 50 of 619
200
200
200
200
200
Got data for city index: 55 of 619
200
200
200
404
Missing data in OpenWeatherAPI for amderma
200
Got data for city index: 60 of 619
200
200
200
200
200
Go

200
Got data for city index: 575 of 619
200
200
404
Missing data in OpenWeatherAPI for gangotri
200
200
Got data for city index: 580 of 619
200
200
200
200
404
Missing data in OpenWeatherAPI for qabis
Got data for city index: 585 of 619
200
200
200
200
200
Got data for city index: 590 of 619
200
200
200
200
200
Got data for city index: 595 of 619
200
200
200
200
200
Got data for city index: 600 of 619
200
200
200
200
200
Got data for city index: 605 of 619
200
200
200
200
200
Got data for city index: 610 of 619
200
200
200
200
200
Got data for city index: 615 of 619
200
200
200


,City,Latitude,Longitude,Temperature,Feels Like,Humidity,Pressure,Wind Speed,Cloudiness
0,Vaini,-21.20,-175.20,78.80,82.33,83,1015,10.29,20
1,Lakes Entrance,-37.88,147.98,67.96,63.32,47,1015,7.00,0
2,Bethel,41.37,-73.41,28.18,15.91,68,1006,12.75,90
3,Kapaa,22.08,-159.32,75.29,74.03,74,1018,12.75,40
4,Ushuaia,-54.80,-68.30,44.60,38.39,93,1006,8.05,75


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [8]:
df.to_csv(output_data_file, index=False)

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression